<a href="https://colab.research.google.com/github/RVanga2005/RAP_EMCC_Code/blob/main/Bin_LogisticReg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from statsmodels.stats.outliers_influence import variance_inflation_factor

#Takes in pandas dataframe (df), lists of categorical and continuous predictors to check, and the output variable column name
def binary_logistic_regression(df, categorical_vars, continuous_vars, output_var):

    # Validate input
    all_cols = categorical_vars + continuous_vars + [output_var]
    missing_cols = [col for col in all_cols if col not in df.columns]

    if missing_cols:
        raise ValueError(f"Columns not found in dataframe: {missing_cols}")

    # Validate output variable is binary
    if df[output_var].nunique(dropna=False) != 2:
        raise ValueError(f"Output variable '{output_var}' must be binary.")

    #May need to tweak encoding of cat/cont variables to create new columns, rather than mutating original value

    # One-hot encode categorical variables to avoid multicollinearity
    df_encoded = pd.get_dummies(df, columns=categorical_vars, drop_first=True)

    # Standardize (mean = 0, var = 1) continuous variables
    df_encoded[continuous_vars] = StandardScaler().fit_transform(df_encoded[continuous_vars])

    # Prepare model input
    X = df_encoded.drop(columns=[output_var])
    X = sm.add_constant(X)  # Add intercept - logistic regression requires intercept
    y = df_encoded[output_var]

    # Fit logistic regression model
    model = sm.Logit(y, X).fit(disp=0)

    # Extract model summary stats
    summary_df = model.summary2().tables[1]  # Coefficients table
    summary_df = summary_df.rename(columns={'Coef.': 'Coef', 'Std.Err.': 'SE Coef', 'z': 'Z-Value', 'P>|z|': 'P-Value'})

    # Compute Variance Inflation Factor (VIF) - checks for multicollinearity
    vif_data = pd.DataFrame()
    vif_data['Variable'] = X.columns
    vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    # Compute Odds Ratios and 95% Confidence Intervals (converts from log form to odds ratios)
    summary_df['Odds Ratio'] = np.exp(summary_df['Coef'])
    summary_df['95% CI Lower'] = np.exp(summary_df['Coef'] - 1.96 * summary_df['SE Coef'])
    summary_df['95% CI Upper'] = np.exp(summary_df['Coef'] + 1.96 * summary_df['SE Coef'])

    # Merge VIF values
    summary_df = summary_df.merge(vif_data, left_index=True, right_on="Variable").drop(columns=['Variable'])

    # Compute Model Fit Metrics
    deviance_rsq = 1 - (model.deviance / model.null_deviance)
    n = len(y)
    numPredictors = X.shape[1] - 1  # Number of predictors excluding intercept
    deviance_rsq_adj = 1 - ((model.deviance - numPredictors) / (model.null_deviance - 1))

    # AIC, AICc, BIC
    aic = model.aic
    aicc = aic + (2 * k * (k + 1)) / (n - k - 1) if n > k + 1 else np.nan
    bic = model.bic

    # Compute Area Under ROC Curve
    auc = roc_auc_score(y, model.predict(X))

    # Print Model Fit Metrics
    print(f"Deviance R-Sq: {deviance_rsq:.4f}")
    print(f"Deviance R-Sq (Adj): {deviance_rsq_adj:.4f}")
    print(f"AIC: {aic:.4f}")
    print(f"AICc: {aicc:.4f}")
    print(f"BIC: {bic:.4f}")
    print(f"Area Under ROC Curve: {auc:.4f}")

    return summary_df